In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

# Grafiklerin notebook içinde görünmesi için
%matplotlib inline

# Bilimsel gösterim ayarları (Virgülden sonra 2 hane)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

def clean_and_load_hwinfo(filepath):
    """
    HWInfo CSV dosyasını okur, boolean dönüşümlerini yapar ve 
    DateTime indekslemesi oluşturur.
    """
    # 1. Dosyayı oku (Olası 'low_memory' uyarısını önlemek için tam okuma)
    # on_bad_lines='skip' ile hatalı satırları atlarız, genelde dosya sonlarında boş satır olabilir.
    df = pd.read_csv(filepath, low_memory=False, on_bad_lines='skip')
    
    # 2. Gereksiz boşlukları temizle (Sütun isimlerinde)
    df.columns = [c.strip() for c in df.columns]
    
    # 3. Boolean Dönüşüm Sözlüğü
    # HWInfo dil ayarlarına göre Türkçe veya İngilizce olabilir.
    bool_mapping = {
        'Evet': True, 'Yes': True, 
        'Hayır': False, 'No': False, 
        'Yok': False, 'None': False,
        'Var': True
    }
    
    # 4. Tüm sütunları gez ve dönüşümü uygula
    for col in df.columns:
        # Eğer sütun object (string) tipindeyse kontrol et
        if df[col].dtype == 'object':
            # Benzersiz değerlere bak, eğer mapping içindekilerle eşleşiyorsa dönüştür
            unique_vals = set(df[col].dropna().unique())
            if unique_vals.issubset(set(bool_mapping.keys())):
                df[col] = df[col].map(bool_mapping)
    
    # 5. Tarih ve Saat Birleştirme (Time Series Index)
    # 'Date' ve 'Time' sütunlarının varlığını kontrol et
    if 'Date' in df.columns and 'Time' in df.columns:
        try:
            # Tarih formatı 29.11.2025 şeklinde gelmiş, dayfirst=True önemli.
            df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
            df.set_index('Datetime', inplace=True)
            # Orijinal Date/Time sütunlarını düşürebiliriz veya tutabiliriz. Şimdilik tutalım.
        except Exception as e:
            print(f"Tarih formatlama hatası ({filepath}): {e}")
            
    # 6. Sayısal olması gereken ancak string kalmış sütunları zorla (Errors='coerce' ile)
    # Örneğin bazen voltaj değerleri string gelebilir.
    # Sayısal olmayan sütunları (Date, Time hariç) float'a çevirmeyi dene
    cols_to_exclude = ['Date', 'Time']
    for col in df.columns:
        if col not in cols_to_exclude and df[col].dtype == 'object':
            # Eğer sütun tamamen boolean'a dönüşmediyse numeric yapmayı dene
            if not pd.api.types.is_bool_dtype(df[col]):
                 df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

print("Fonksiyonlar tanımlandı.")

In [ ]:
# Dosya yolları (Dosya isimleri sizin yüklediklerinizle aynı olmalı)
files = {
    "Default_Optimized": "Fans ve apu down.csv",
    "PBO_95W_Neg30_Plus200": "-30 CCD PBO 95 +200 mhz.csv",
    "PBO_65W_Neg25": "65W-85W_75A_150A-25_Per_CCD.csv",
    "PBO_95W_Neg20": "95W-115W_80A_140A-20_Per_CCD.csv"
}

dataframes = {}

print("Veriler yükleniyor...")
for name, path in files.items():
    if os.path.exists(path):
        df = clean_and_load_hwinfo(path)
        dataframes[name] = df
        print(f"[{name}] yüklendi. Satır sayısı: {len(df)}")
    else:
        print(f"HATA: {path} dosyası bulunamadı!")

print("\nTüm veriler işlendi.")

In [ ]:
def analyze_ram_timings(data_dict):
    """
    Yüklenen dataframe'ler içindeki RAM zamanlamalarını (Tcas, Trcd vb.) raporlar.
    """
    # RAM ile ilgili anahtar kelimeler (HWInfo genelde [T] birimi kullanır)
    timing_keywords = ['Tcas', 'Trcd', 'Trp', 'Tras', 'Trc', 'Trfc', 'Command Rate']
    
    print("--- RAM Timing Analizi ---")
    
    for name, df in data_dict.items():
        print(f"\nProfil: {name}")
        found_timings = []
        for col in df.columns:
            # Sütun isminde keyword geçiyor mu kontrol et
            if any(k.lower() in col.lower() for k in timing_keywords):
                # İlk geçerli değeri al (Genelde sabittir)
                val = df[col].iloc[0] if len(df) > 0 else "N/A"
                found_timings.append((col, val))
        
        if found_timings:
            for timing, value in found_timings:
                print(f"  {timing}: {value}")
        else:
            print("  RAM Timing verisi bulunamadı.")

analyze_ram_timings(dataframes)